In [1]:
%load_ext autoreload
%load_ext autotime
%autoreload 2

time: 594 µs (started: 2021-11-18 13:10:11 -08:00)


# Options Apps Project

## Goal
Creating a suite of equity option web apps which expands on, and was inspired by, [Harshit Tyagi’s](https://www.linkedin.com/in/tyagiharshit/) [article](https://www.kdnuggets.com/2021/09/-structured-financial-newsfeed-using-python-spacy-and-streamlit.html) on building a financial newsfeed app.

## Skills Involved
Web scraping, NLP, options price modeling, predictive modeling of volitility, and building apps with Streamlit.

## Imports

In [2]:
import sys, os, math
import requests
from bs4 import BeautifulSoup
import spacy
from spacy import displacy
import yfinance as yf
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime
import time

#  setting path
gparent = os.path.join(os.pardir)
sys.path.append(gparent)

from src import helper_functions as h

time: 13.2 s (started: 2021-11-18 13:10:11 -08:00)


## requests: Grabbing Headlines

In [3]:
# grabbing headlines
r = requests.get("http://feeds.marketwatch.com/marketwatch/topstories/")
r

<Response [200]>

time: 225 ms (started: 2021-11-18 13:10:24 -08:00)


## bs4: Saving Headlines

In [4]:
# saving headlines to a list
soup = BeautifulSoup(r.content, features='lxml')
headlines = soup.findAll('title')
print(f'{type(headlines)}\n')
print(headlines)

<class 'bs4.element.ResultSet'>

[<title>MarketWatch.com - Top Stories</title>, <title>MarketWatch.com - Top Stories</title>, <title>: Ford CEO touts auto maker’s EV plans: ‘Whatever it takes, find a way’</title>, <title>Distributed Ledger: Some crypto companies frown on infrastructure law, while some financial advisors cheer for it</title>, <title>The Tell: Should stock-market investors who ‘missed the rally’ buy now? Here’s what UBS says</title>, <title>Bond Report: Treasury yields extend declines, with 10- and 30-year posting biggest two-day drop in over a week</title>, <title>Market Extra: The lower-for-longer camp on U.S. bond yields is standing firmly by its views heading into 2022</title>, <title>The Ratings Game: Activision Blizzard stock downgraded as misconduct charges bring a ‘significant amount of uncertainty’</title>, <title>: A mostly Black Alabama county has no municipal sewer service. Can the 1964 Civil Rights Act be used for environmental justice?</title>, <title>: Lob

### Grabbing Headline String
Grabbing headline and printing headline and isolated text.

In [5]:
# grabbing test headline
test_headline = headlines[2]
print(f'{test_headline}\n')
print(test_headline.text)

<title>: Ford CEO touts auto maker’s EV plans: ‘Whatever it takes, find a way’</title>

: Ford CEO touts auto maker’s EV plans: ‘Whatever it takes, find a way’
time: 881 µs (started: 2021-11-18 13:10:25 -08:00)


## spaCy: Tests
Testing tokenization and name extraction on a single string.

### Loading Model

In [6]:
# loading spacy model
nlp = spacy.load('en_core_web_sm')

time: 640 ms (started: 2021-11-18 13:10:25 -08:00)


### Tokenizing String
Tokenizing the test headline.

In [7]:
# checking the test case
processed_hline = nlp(test_headline.text)
print(f'{test_headline}\n')
for token in processed_hline:
  print(token)

<title>: Ford CEO touts auto maker’s EV plans: ‘Whatever it takes, find a way’</title>

:
Ford
CEO
touts
auto
maker
’s
EV
plans
:
‘
Whatever
it
takes
,
find
a
way
’
time: 11.2 ms (started: 2021-11-18 13:10:26 -08:00)


## Saving List of Tokenized Headlines

In [8]:
processed_hlines = [nlp(headlines[i].text) for i in range(len(headlines))]
for line in processed_hlines:
    print(line)

MarketWatch.com - Top Stories
MarketWatch.com - Top Stories
: Ford CEO touts auto maker’s EV plans: ‘Whatever it takes, find a way’
Distributed Ledger: Some crypto companies frown on infrastructure law, while some financial advisors cheer for it
The Tell: Should stock-market investors who ‘missed the rally’ buy now? Here’s what UBS says
Bond Report: Treasury yields extend declines, with 10- and 30-year posting biggest two-day drop in over a week
Market Extra: The lower-for-longer camp on U.S. bond yields is standing firmly by its views heading into 2022
The Ratings Game: Activision Blizzard stock downgraded as misconduct charges bring a ‘significant amount of uncertainty’
: A mostly Black Alabama county has no municipal sewer service. Can the 1964 Civil Rights Act be used for environmental justice?
: Lobbyist for truckers on the ‘driver shortage’: ‘If the job that you’re offering sucks,
is the solution really go find more suckers, or should you improve the job?’
: Starbucks and Amazon 

## Getting Org Names From Headlines
Visualizing named entities (real world objects) in the headlines and creating a set of organiziations.

In [9]:
# pulling company name tokens from headlines
companies = []
for title in processed_hlines:
    doc = nlp(title.text)
    if len(doc.ents) != 0:
        displacy.render(doc, style='ent')
    else:
        pass
    for token in doc.ents:
        if token.label_ == 'ORG':
            companies.append(token.text)
        else:
            pass
companies = set(companies)
print(companies)

{'Starbucks', 'Ford', 'UBS', 'EV', 'Treasury', 'Amazon'}
time: 132 ms (started: 2021-11-18 13:10:26 -08:00)


## Scraping S&P 500 Stock Table w/Requests & BeautifulSoup

In [10]:
# scraping S&P wikipedia page
r = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')

# parsing the html
soup = BeautifulSoup(r.text, 'lxml')

# extracting the table
table = soup.find('table', {'class': 'wikitable sortable'})

# printing row with first stock
print(table.findAll('tr')[1:2])

[<tr>
<td><a class="external text" href="https://www.nyse.com/quote/XNYS:MMM" rel="nofollow">MMM</a>
</td>
<td><a href="/wiki/3M" title="3M">3M</a></td>
<td><a class="external text" href="https://www.sec.gov/edgar/browse/?CIK=66740" rel="nofollow">reports</a></td>
<td>Industrials</td>
<td>Industrial Conglomerates</td>
<td><a href="/wiki/Saint_Paul,_Minnesota" title="Saint Paul, Minnesota">Saint Paul, Minnesota</a></td>
<td>1976-08-09</td>
<td>0000066740</td>
<td>1902
</td></tr>]
time: 756 ms (started: 2021-11-18 13:10:26 -08:00)


### Ticker Symbols
Grabbing the ticker symbol from the first cell of each row.

In [11]:
# making list of symbols
symbols = [row.findAll('td')[0].text for row in table.findAll('tr')[1:]]

# checking length
print(f'List Length: {len(symbols)} \n')

# checking first 5 symbols
print(f'First five symbols: {symbols[:5]}')

List Length: 505 

First five symbols: ['MMM\n', 'ABT\n', 'ABBV\n', 'ABMD\n', 'ACN\n']
time: 16.3 ms (started: 2021-11-18 13:10:27 -08:00)


The length of the list looks good, but we have extra characters we need to strip from the symbols.

In [12]:
# stripping new line character from the strings 
symbols = list(map(lambda s: s.strip(), symbols))

# checking first 5 symbols
print(symbols[:5])

['MMM', 'ABT', 'ABBV', 'ABMD', 'ACN']
time: 1.47 ms (started: 2021-11-18 13:10:27 -08:00)


Looks good. We can apply the same technique to the second cell of the row to scrape and save the company name, and to the fifth cell to grab the company industry. 

### Names
Grabbing the company name from the second cell of each row.

In [13]:
# making list of names
names = [row.findAll('td')[1].text for row in table.findAll('tr')[1:]]

# checking first five names
print(names[:5])

['3M', 'Abbott Laboratories', 'AbbVie', 'Abiomed', 'Accenture']
time: 15.2 ms (started: 2021-11-18 13:10:27 -08:00)


## S&P 500 Dataframe
Creating a data frame of stocks in the S&P 500 index.

In [14]:
# making a data dictionary
data = {'Company Name': names, 'Symbol': symbols}

# creating data frame from the data
stocks_df = pd.DataFrame.from_dict(data)

# checking shape and first five rows
print(stocks_df.shape)
stocks_df.head()

(505, 2)


,Company Name,Symbol
0,3M,MMM
1,Abbott Laboratories,ABT
2,AbbVie,ABBV
3,Abiomed,ABMD
4,Accenture,ACN


time: 18.7 ms (started: 2021-11-18 13:10:27 -08:00)


### Checking yf Stock Info Dictionary Keys
Checking the dictionary keys available for pulling stock information.

In [15]:
# instantiating a ticker object
ACN = yf.Ticker('ACN')

time: 4.16 ms (started: 2021-11-18 13:10:27 -08:00)


In [16]:
# checking keys
ACN.info.keys()

dict_keys(['zip', 'sector', 'fullTimeEmployees', 'longBusinessSummary', 'city', 'phone', 'country', 'companyOfficers', 'website', 'maxAge', 'address1', 'fax', 'industry', 'address2', 'ebitdaMargins', 'profitMargins', 'grossMargins', 'operatingCashflow', 'revenueGrowth', 'operatingMargins', 'ebitda', 'targetLowPrice', 'recommendationKey', 'grossProfits', 'freeCashflow', 'targetMedianPrice', 'currentPrice', 'earningsGrowth', 'currentRatio', 'returnOnAssets', 'numberOfAnalystOpinions', 'targetMeanPrice', 'debtToEquity', 'returnOnEquity', 'targetHighPrice', 'totalCash', 'totalDebt', 'totalRevenue', 'totalCashPerShare', 'financialCurrency', 'revenuePerShare', 'quickRatio', 'recommendationMean', 'exchange', 'shortName', 'longName', 'exchangeTimezoneName', 'exchangeTimezoneShortName', 'isEsgPopulated', 'gmtOffSetMilliseconds', 'quoteType', 'symbol', 'messageBoardId', 'market', 'annualHoldingsTurnover', 'enterpriseToRevenue', 'beta3Year', 'enterpriseToEbitda', '52WeekChange', 'morningStarRiskR

time: 2.75 s (started: 2021-11-18 13:10:27 -08:00)


### Data for Stocks in the News
Creating a data frame of price and dividend information for S&P 500 stocks in the news in the following steps:

- Create a data dictionary from the list of companies in the headlines.

- Create a data frame from data dictionary.

In [17]:
# creating empty stock info dictionary
stock_data = {
    'Company': [],
    'Symbol': [],
    'currentPrice': [],
    'dayHigh': [],
    'dayLow': [],
    '52wkHigh': [],
    '52wkLow': [],
    'dividendRate': []
    
}

# loading stocks from s&p dataframe and appending data from yf
for company in companies:

    try:
        if stocks_df['Company Name'].str.contains(company).sum():
            symbol = stocks_df[stocks_df['Company Name'].\
                                str.contains(company)]['Symbol'].values[0]
            org_name = stocks_df[stocks_df['Company Name'].\
                                str.contains(company)]['Company Name'].values[0]
            stock_data['Company'].append(org_name)
            stock_data['Symbol'].append(symbol)
            stock_info = yf.Ticker(symbol).info
            stock_data['currentPrice'].append(stock_info['currentPrice'])
            stock_data['dayHigh'].append(stock_info['dayHigh'])
            stock_data['dayLow'].append(stock_info['dayLow'])
            stock_data['52wkHigh'].append(stock_info['fiftyTwoWeekHigh'])
            stock_data['52wkLow'].append(stock_info['fiftyTwoWeekLow'])            
            
            # converting dividend None types to floats
            dividend = stock_info['dividendRate']
            if dividend != None:
                dividend = dividend
            else:
                dividend = 0
            stock_data['dividendRate'].append(dividend)
        else:
            pass
    except:
        pass

time: 14.7 s (started: 2021-11-18 13:10:30 -08:00)


In [18]:
# checking dict
stock_data

{'Company': ['Starbucks', 'Ford', 'Amazon'],
 'Symbol': ['SBUX', 'F', 'AMZN'],
 'currentPrice': [112.9, 19.56, 3696.06],
 'dayHigh': [113.2299, 20.26, 3704.2],
 'dayLow': [111.34, 19.36, 3561.04],
 '52wkHigh': [126.32, 20.51, 3773.08],
 '52wkLow': [95.92, 8.43, 2881],
 'dividendRate': [1.96, 0.4, 0]}

time: 2.55 ms (started: 2021-11-18 13:10:45 -08:00)


### Data Frame of S&P 500 Stocks in the News

In [19]:
in_the_news = pd.DataFrame(stock_data)
in_the_news.head()

,Company,Symbol,currentPrice,dayHigh,dayLow,52wkHigh,52wkLow,dividendRate
0,Starbucks,SBUX,112.90,113.2299,111.34,126.32,95.92,1.96
1,Ford,F,19.56,20.2600,19.36,20.51,8.43,0.40
2,Amazon,AMZN,3696.06,3704.2000,3561.04,3773.08,2881.00,0.00


time: 7.33 ms (started: 2021-11-18 13:10:45 -08:00)


In [20]:
# checking Dtypes
in_the_news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Company       3 non-null      object 
 1   Symbol        3 non-null      object 
 2   currentPrice  3 non-null      float64
 3   dayHigh       3 non-null      float64
 4   dayLow        3 non-null      float64
 5   52wkHigh      3 non-null      float64
 6   52wkLow       3 non-null      float64
 7   dividendRate  3 non-null      float64
dtypes: float64(6), object(2)
memory usage: 320.0+ bytes
time: 6.2 ms (started: 2021-11-18 13:10:45 -08:00)


## Individual Stock Price Helper Function
Checking the helper function.

In [21]:
h.get_prices('MMM')

,Company,Symbol,Current Price,Intraday High,Intraday Low,52wkHigh,52wkLow,Dividend
0,3M Company,MMM,178.75,182.55,178.7,208.95,163.38,5.92


time: 4.85 s (started: 2021-11-18 13:10:45 -08:00)


In [22]:
h.get_prices('AAPL')

,Company,Symbol,Current Price,Intraday High,Intraday Low,52wkHigh,52wkLow,Dividend
0,Apple Inc.,AAPL,157.87,158.67,153.06,158.67,112.59,0.88


time: 2.59 s (started: 2021-11-18 13:10:50 -08:00)


## Options Markets Tests

In [23]:
# creating a fuction to display available expirations
def get_expirations(symbol : str):
    stock = yf.Ticker(symbol)
    return stock.options

time: 569 µs (started: 2021-11-18 13:10:52 -08:00)


In [24]:
get_expirations('AAPL')

('2021-11-19',
 '2021-11-26',
 '2021-12-03',
 '2021-12-10',
 '2021-12-17',
 '2021-12-23',
 '2021-12-31',
 '2022-01-21',
 '2022-02-18',
 '2022-03-18',
 '2022-04-14',
 '2022-05-20',
 '2022-06-17',
 '2022-09-16',
 '2023-01-20',
 '2023-03-17',
 '2023-06-16',
 '2023-09-15',
 '2024-01-19')

time: 188 ms (started: 2021-11-18 13:10:52 -08:00)


In [25]:
get_expirations('MMM')

('2021-11-19',
 '2021-11-26',
 '2021-12-03',
 '2021-12-10',
 '2021-12-17',
 '2021-12-23',
 '2021-12-31',
 '2022-01-21',
 '2022-04-14',
 '2022-06-17',
 '2022-07-15',
 '2023-01-20',
 '2024-01-19')

time: 146 ms (started: 2021-11-18 13:10:53 -08:00)


In [26]:
def options_mrkt_test(symbol : str, expiration : str, option : str):
    stock = yf.Ticker(symbol)
    current_price = stock.info['currentPrice']
    dividend = stock.info['dividendRate']
    url = ('https://www.treasury.gov/resource-center/'
    'data-chart-center/interest-rates/Pages/TextView.aspx?data=yield')
    rates = pd.read_html(url)
    risk_free_rate = rates[1]['3 mo']
    print(f"Current Price: ${current_price}")
    print(f"Dividend: ${dividend}")
    print(f"3 Month TBill Rate: {risk_free_rate.to_string().split()[1]}%")
    opt = stock.option_chain(expiration)
    if option == 'calls':
        return opt.calls.sort_values(by='strike')
    elif option == 'puts':
        return opt.puts.sort_values(by='strike')

time: 1.22 ms (started: 2021-11-18 13:10:53 -08:00)


In [28]:
options_mrkt_test('MMM', '2021-11-19', 'calls' )

Current Price: $178.77
Dividend: $5.92
3 Month TBill Rate: 0.05%


,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
0,MMM211119C00100000,2021-11-18 19:20:23,100.0,79.91,78.65,79.35,-0.839996,-1.040243,3.0,1,3.718751,True,REGULAR,USD
1,MMM211119C00110000,2021-11-17 20:15:45,110.0,73.58,68.40,69.95,0.000000,0.000000,6.0,0,3.468751,True,REGULAR,USD
2,MMM211119C00115000,2021-11-17 20:36:45,115.0,68.95,63.00,65.40,0.000000,0.000000,50.0,0,3.224611,True,REGULAR,USD
3,MMM211119C00120000,2021-11-17 20:36:45,120.0,63.70,57.70,60.90,0.000000,0.000000,55.0,0,3.070315,True,REGULAR,USD
4,MMM211119C00125000,2021-11-17 20:15:47,125.0,58.58,53.60,55.15,0.000000,0.000000,4.0,0,2.873050,True,REGULAR,USD
5,MMM211119C00140000,2021-11-17 20:15:49,140.0,43.59,38.45,39.85,0.000000,0.000000,4.0,0,1.910157,True,REGULAR,USD
6,MMM211119C00150000,2021-11-17 20:16:11,150.0,33.60,27.10,30.90,0.000000,0.000000,8.0,0,1.318363,True,REGULAR,USD
7,MMM211119C00155000,2021-11-17 19:53:16,155.0,28.00,23.80,24.55,0.000000,0.000000,60.0,1,1.247074,True,REGULAR,USD
8,MMM211119C00160000,2021-11-17 20:16:12,160.0,23.60,17.90,20.30,0.000000,0.000000,5.0,1,0.976563,True,REGULAR,USD
9,MMM211119C00162500,2021-11-10 14:30:12,162.5,19.05,14.50,18.30,0.000000,0.000000,1.0,0,0.716800,True,REGULAR,USD


time: 3.41 s (started: 2021-11-18 13:15:52 -08:00)
